# Guided Implementation

## Welcome!

In this project, you'll build a **RAG (Retrieval-Augmented Generation)** system that can answer questions about your documents.

### What You'll Learn:
- ✅ File I/O (reading documents)
- ✅ String manipulation (text chunking)
- ✅ Functions and parameters
- ✅ Lists and dictionaries
- ✅ Loops and conditionals
- ✅ Basic calculations and statistics

### What's Provided for You:
- ✅ Embedding model (converts text to numbers)
- ✅ Vector database (stores and searches embeddings)
- ✅ LLM connection (generates answers)

Let's get started! 🚀

---
## Setup: Import Libraries

In [ ]:
# Import the pre-built helper module
from rag_helpers import (
    EmbeddingModel,
    VectorDatabase,
    LLM,
    Timer,
    print_separator,
    print_search_results,
    print_rag_answer,
    check_setup
)

# Import standard Python libraries you'll use
from pathlib import Path
from typing import List, Dict
import json

# Check if everything is installed correctly
check_setup()

---
## Configuration

Set up the basic settings for your RAG system.

In [ ]:
DOCS_FOLDER = "./team_docs_test"

CHUNK_SIZE = 500     
OVERLAP = 50          
TOP_K = 3

print(f"Configuration:")
print(f"  Documents folder: {DOCS_FOLDER}")
print(f"  Chunk size: {CHUNK_SIZE} characters")
print(f"  Overlap: {OVERLAP} characters")
print(f"  Top-K results: {TOP_K}")

---
## Document Loading

**What to do:**
1. Loop through all `.txt` files in the folder
2. Read each file's content
3. Store the content and filename in a dictionary
4. Return a list of these dictionaries

**Python concepts:** File I/O, loops, dictionaries, lists

In [ ]:
def load_documents(folder_path: str) -> List[Dict[str, str]]:
    """
    Load all text documents from a folder.

    Args:
        folder_path: Path to folder containing .txt files

    Returns:
        List of dictionaries, each containing:
        - 'content': the text content of the file
        - 'filename': the name of the file

    Example:
        [
            {'content': 'This is doc 1...', 'filename': 'doc1.txt'},
            {'content': 'This is doc 2...', 'filename': 'doc2.txt'}
        ]
    """


    documents = [] 

    folder = Path(folder_path)

    for file_path in folder.glob("*.txt"):
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        doc_dict = {
            'content': content,
            'filename': file_path.name
        }
        documents.append(doc_dict)

    print(f"✓ Loaded {len(documents)} documents")
    return documents


documents = load_documents(DOCS_FOLDER)

if documents:
    print(f"\nFirst document: {documents[0]['filename']}")
    print(f"Content preview: {documents[0]['content'][:200]}...")
else:
    print("⚠️  No documents loaded! Check your folder path.")



---
## Text Chunking Function

**Why?** Long documents are too big for embeddings. We need to split them into smaller pieces.

**What to do:**
1. Start at the beginning of the text
2. Take a chunk of `chunk_size` characters
3. Move forward by `chunk_size - overlap` characters
4. Repeat until you reach the end

**Python concepts:** String slicing, loops, lists

In [ ]:
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    
    chunks = []  
    position = 0 

    step = chunk_size - overlap
    if step <= 0:
        raise ValueError('chunk_size must be greater than overlap.')
        
    while position < len(text):
        chunk = text[position : position + chunk_size]
        chunks.append(chunk)
        position += step

    return chunks


test_text = "This is a test. " * 50  # Create a long test string
test_chunks = chunk_text(test_text, chunk_size=100, overlap=20)

print(f"Test text length: {len(test_text)} characters")
print(f"Number of chunks: {len(test_chunks)}")
print(f"\nFirst chunk: {test_chunks[0]}")
if len(test_chunks) > 1:
    print(f"Second chunk: {test_chunks[1]}")

---
## Process All Documents into Chunks

**What to do:**
1. Loop through each document
2. Chunk the document's content
3. For each chunk, create metadata (which file it came from, which chunk number)
4. Store everything in a list

**Python concepts:** Nested loops, dictionaries, enumerate

In [ ]:
def process_documents(documents: List[Dict[str, str]],
                     chunk_size: int = 500,
                     overlap: int = 50) -> tuple:
    """
    Process all documents into chunks with metadata.

    Args:
        documents: List of document dictionaries
        chunk_size: Size of each chunk
        overlap: Overlap between chunks

    Returns:
        Tuple of (chunk_texts, chunk_metadatas)
        - chunk_texts: List of chunk strings
        - chunk_metadatas: List of metadata dictionaries
    """


    chunk_texts = []
    chunk_metadatas = []

    for doc in documents:
        content = doc['content']
        filename = doc['filename']
        chunks = chunk_text(content, chunk_size, overlap)

        for idx, chunk in enumerate(chunks):
            chunk_texts.append(chunk)
            metadata = {
                'source': filename,
                'chunk_id': idx
            }
            chunk_metadatas.append(metadata)

    print(f"✓ Created {len(chunk_texts)} chunks from {len(documents)} documents")
    return chunk_texts, chunk_metadatas

chunk_texts, chunk_metadatas = process_documents(documents, CHUNK_SIZE, OVERLAP)

if chunk_texts:
    print(f"\nExample chunk:")
    print(f"  Source: {chunk_metadatas[0]['source']}")
    print(f"  Chunk ID: {chunk_metadatas[0]['chunk_id']}")
    print(f"  Text: {chunk_texts[0][:200]}...")

---
## Pre-Built: Create Embeddings and Store in Database

This part uses the pre-built helpers. Just run these cells - no coding needed! ✨

In [ ]:
# Initialize the embedding model (pre-built)
print("Initializing embedding model...")
embedder = EmbeddingModel()

# Create embeddings for all chunks (pre-built)
print("\nCreating embeddings...")
embeddings = embedder.embed_multiple(chunk_texts)
print(f"✓ Created {len(embeddings)} embeddings")

In [ ]:
# Initialize vector database (pre-built)
print("Initializing vector database...")
vector_db = VectorDatabase()

# Add chunks to database (pre-built)
print("\nAdding chunks to database...")
vector_db.add_chunks(chunk_texts, embeddings, chunk_metadatas)

In [ ]:
# Initialize LLM connection (pre-built)
print("Connecting to Ollama LLM...")
llm = LLM(model='gemma3:1b-it-qat')

# Test the connection
print("\nTesting LLM connection...")
if llm.test_connection():
    print("✓ LLM is working!")
else:
    print("⚠️  LLM connection failed! Make sure Docker container is running.")

---
## RAG Query Function

**What to do:**
1. Embed the user's question
2. Search the database for similar chunks
3. Build a prompt with the retrieved context
4. Ask the LLM to answer based on the context
5. Return the answer and metadata

**Python concepts:** Functions, string formatting, dictionaries

In [ ]:
def rag_query(question: str, top_k: int = 3) -> Dict:
    """
    Answer a question using RAG (Retrieval-Augmented Generation).

    Args:
        question: The user's question
        top_k: How many chunks to retrieve

    Returns:
        Dictionary with:
        - 'question': the original question
        - 'answer': the LLM's answer
        - 'sources': list of source filenames
        - 'contexts': list of retrieved chunks
        - 'time': how long it took
    """

    timer = Timer()
    timer.start()

    # Step 1: Embed question
    query_embedding = embedder.embed_text(question)  

    # Step 2: Search database
    results = vector_db.search(query_embedding, top_k=top_k)  

    # Step 3: Extract results
    retrieved_chunks = results['documents'][0] 
    retrieved_metadata = results['metadatas'][0] 

    # Step 4: Build context
    context = "\n\n".join(retrieved_chunks) 

    # Step 5: Create prompt (use this template)
    prompt = f"""Answer the question based on the context below.

Context:
{context}

Question: {question}

Answer:"""

    # Step 6: Generate answer
    answer = llm.generate_answer(prompt) 

    # Step 7: Extract sources
    sources = [meta.get("source", "unknown") for meta in retrieved_metadata]  

    # Stop timer
    elapsed_time = timer.stop()

    # Step 8: Return results
    return {
        'question': question,
        'answer': answer,
        'sources': sources,
        'contexts': retrieved_chunks,
        'time': elapsed_time
    }


print("✓ RAG query function defined!")

---
## Test Your RAG System!

Let's try asking some questions!

In [ ]:
# Test question 1
result = rag_query("When did the Big Bang occur, according to current measurements?")
# expected_answer: Approximatedly 13.8 billion years

# Pretty print the result
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

In [ ]:
# Test question 2
result = rag_query("What does the cosmological principle state about the universe?")
# expected_answer: The cosmological principle states that on sufficiently large scales, the universe appears homogeneous and isotropic

# Pretty print the result
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

In [ ]:
# Test question 3
result = rag_query("Which observation reveals that galaxies recede from Earth at speeds proportional to their distance?")
# expected_answer: The cosmological redshift.

# Pretty print the result
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

In [ ]:
# Test question 4
result = rag_query("How does the cosmological principle support the use of large-scale surveys when testing models of the universe?")
# expected_answer: The cosmological principle assumes that the universe is homogeneous and isotropic on large scales, 
# which means that observations from one region can be generalized to others. 
# This supports the use of large-scale surveys because data gathered from a limited portion of 
# the sky can still provide valid tests of cosmological models that describe the entire universe.

# Pretty print the result
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

In [ ]:
# Test question 5
result = rag_query("If the cosmological principle were violated on large scales, what implications would this have for current cosmological models and observations?")
# expected_answer: A violation of the cosmological principle would undermine the foundations of many standard cosmological models,
#  which assume large-scale homogeneity and isotropy. It would imply that observations from Earth may not be representative 
# of the universe as a whole, requiring revised models, reinterpretation of observational data, and potentially 
# new physics to explain variations in matter distribution or directional differences.

print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

In [ ]:
# Test question 6
my_question = "Describe the complete sequence of events from the end of cosmic inflation to the moment of recombination, including how quantum fluctuations grow into large-scale structure, how baryon acoustic oscillations develop, and how matter–radiation decoupling produces the features observed in the cosmic microwave background." 
# expected_answer: After inflation ends, the universe undergoes reheating, during which the vacuum energy that drove inflation decays 
# .......
#  of the sound horizon becomes imprinted as baryon acoustic oscillations in the matter distribution…

result = rag_query(my_question)
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

In [ ]:
# Test question 7
result = rag_query("What observational evidence first confirmed the existence of cosmic inflation?")
# expected_answer: The first strong observational evidence for cosmic inflation is the BICEP2 experiment's detection of B-mode 
# polarization in the Cosmic Microwave Background (CMB), a twisting pattern in the light left over from the Big Bang, 
# which is the signature of primordial gravitational waves generated during the universe's rapid inflation.

# Pretty print the result
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

In [ ]:
# Test question 8
result = rag_query("What telescope first detected the B-mode polarization pattern that would serve as direct evidence for cosmic inflation?")
# expected_answer: The BICEP2 telescope

# Pretty print the result
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

In [ ]:
# Test question 9
result = rag_query("Why is it flat?")
# expected_answer: The BICEP2 telescope

# Pretty print the result
print_rag_answer(
    result['question'],
    result['answer'],
    result['sources'],
    result['time']
)

---
## Create Test Dataset

**What to do:**
1. Think of 10 questions your documents can answer
2. For each question, write the expected answer
3. Store them in a structured format

**Python concepts:** Lists, dictionaries, data structures

In [ ]:
test_questions = [
    # Example (replace with your own!):
    {
        "question": "Which observation reveals that galaxies recede from Earth at speeds proportional to their distance?",
        "expected_answer": "Hubble's law, which shows a linear relationship between distance and recessional velocity.",
        "category": "factual"
    },
    {
        "question": "What discovery in 1964 provided a snapshot of the universe 380,000 years after the Big Bang?",
        "expected_answer": "The cosmic microwave background radiation, discovered by Penzias and Wilson.",
        "category": "factual"
    },
    {
        "question": "Which early-universe process produced helium-4, deuterium, helium-3, and lithium-7 within minutes of the Big Bang?",
        "expected_answer": "Big Bang nucleosynthesis.",
        "category": "factual"
    },
    {
        "question": "What cosmological principle states that the universe is homogeneous and isotropic on large scales?",
        "expected_answer": "The cosmological principle.",
        "category": "factual"
    },
    {
        "question": "Which problem in cosmology concerns the extreme uniformity of the cosmic microwave background in regions that should not have been in causal contact?",
        "expected_answer": "The horizon problem.",
        "category": "factual"
    },
    {
        "question": "What is the term for the early rapid expansion that helps explain the universe's flatness and uniformity?",
        "expected_answer": "Cosmic inflation.",
        "category": "factual"
    },
    {
        "question": "What phenomenon shows that the universe's expansion is accelerating?",
        "expected_answer": "Observations of Type Ia supernovae indicating cosmic acceleration.",
        "category": "factual"
    },
    {
        "question": "Which component of the universe accounts for roughly 27 percent of its total mass-energy and interacts only through gravity?",
        "expected_answer": "Dark matter.",
        "category": "factual"
    },
    {
        "question": "During which epoch did quarks combine to form protons and neutrons?",
        "expected_answer": "Around 10⁻⁶ seconds after the Big Bang, during early cooling.",
        "category": "factual"
    },
    {
        "question": "What event made the universe transparent and allowed photons to travel freely, forming today's cosmic microwave background?",
        "expected_answer": "Recombination, when electrons and nuclei formed neutral atoms.",
        "category": "factual"
    }
]



print(f"✓ Created {len(test_questions)} test questions")
print(f"\nExample question:")
print(f"  Q: {test_questions[0]['question']}")
print(f"  Expected: {test_questions[0]['expected_answer']}")

---
## Calculate Evaluation Metrics

**Python concepts:** Functions, calculations, statistics

In [ ]:
def calculate_average_latency(results: List[Dict]) -> float:
    """
    Calculate average response time.

    Args:
        results: List of result dictionaries (each has 'time' field)

    Returns:
        Average time in seconds
    """


    if not results:
        return 0.0
    
    total_time = 0.0
    for r in results:
        total_time += r.get("time", 0)

    return total_time / len(results)



def count_successful_retrievals(results: List[Dict]) -> int:
    """
    Count how many queries successfully retrieved context.

    Args:
        results: List of result dictionaries

    Returns:
        Number of successful retrievals
    """


    count = 0
    for r in results:
        contexts = r.get("contexts", [])
        if contexts:  
            count += 1

    return count



def get_all_sources(results: List[Dict]) -> List[str]:
    """
    Get unique list of all sources used.

    Args:
        results: List of result dictionaries

    Returns:
        List of unique source filenames
    """


    all_sources = set()
    for r in results:
        sources = r.get("sources", [])
        for s in sources:
            all_sources.add(s)

    return list(all_sources)



print("✓ Evaluation functions defined!")

---
## Run Complete Evaluation

**Python concepts:** Loops, function calls, data aggregation

In [ ]:
def run_evaluation(test_questions: List[Dict]) -> List[Dict]:
    """
    Run RAG system on all test questions.

    Args:
        test_questions: List of test question dictionaries

    Returns:
        List of result dictionaries
    """


    results = []

    for test in test_questions:
        question = test["question"]
        result = rag_query(question)
        results.append(result)

    return results


print("Running evaluation on all test questions...\n")
all_results = run_evaluation(test_questions)

print(f"\n✓ Completed {len(all_results)} tests")

---
## Display Results

Show the evaluation metrics and results.

In [ ]:
# Calculate metrics using your functions
avg_latency = calculate_average_latency(all_results)
successful_retrievals = count_successful_retrievals(all_results)
all_sources_used = get_all_sources(all_results)
hit_rate = successful_retrievals / len(all_results) if all_results else 0

# Display metrics
print_separator("EVALUATION RESULTS")
print(f"\nTotal Questions Tested: {len(all_results)}")
print(f"Successful Retrievals: {successful_retrievals}")
print(f"Hit Rate: {hit_rate:.2%}")
print(f"Average Latency: {avg_latency:.2f} seconds")
print(f"\nSources Used: {', '.join(all_sources_used)}")
print_separator()

In [ ]:
# Display individual results
print("\nIndividual Test Results:\n")

for i, result in enumerate(all_results, 1):
    print(f"[Test {i}]")
    print(f"Question: {result['question']}")
    print(f"Answer: {result['answer']}") # ['answer'][:200]}...
    print(f"Sources: {', '.join(set(result['sources']))}")
    print(f"Time: {result['time']:.2f}s")
    print("-" * 60)
    print()

---
## Save Your Results

Save your test results to a JSON file for your report.

In [ ]:
# Save results to JSON file
results_summary = {
    'metrics': {
        'total_questions': len(all_results),
        'successful_retrievals': successful_retrievals,
        'hit_rate': hit_rate,
        'average_latency': avg_latency
    },
    'results': all_results
}

with open('evaluation_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("✓ Results saved to 'evaluation_results.json'")

---
## Congratulations! 🎉

You've successfully built a RAG system!

### What You Accomplished:
✅ Loaded documents from files  
✅ Chunked text with overlap  
✅ Created a RAG query pipeline  
✅ Built a test dataset  
✅ Calculated evaluation metrics  
✅ Generated a results report  

### Next Steps:
- Try different chunk sizes and overlaps
- Add more test questions
- Experiment with different values for `top_k`
- Analyze which questions work best
- Write up your findings in a report

Great job! 🚀